## Getting YouTube captions

#### This notebook shows how to gather data from YouTube and create a dataframe with it.

In [2]:
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

In [3]:
api_key = 'API_KEY'
# replace by the API key that you generate on Google Developers

video_id = 'VIDEO_ID'
# replace by the video id which is the part of the URL after 'https://www.youtube.com/watch?v='

candidate = 'CANDIDATE_NAME'
# replace by the name of the candidate

content_type = 'CONTENT_TYPE'
#replace by the content typoe of the video (interview, speech etc.)

In [4]:
youtube = build('youtube', 'v3', developerKey= api_key)
#create a YouTube Data API v.3 object

request = youtube.videos().list(part= "contentDetails,snippet", id = video_id)
response = request.execute()
#make a request to the data of the video and execute it
#the resulting response is a dictionary with lots of metadata of the video

In [5]:
channel_id = response['items'][0]['snippet']['channelId']
channel_title = response['items'][0]['snippet']['channelTitle']
video_id = response['items'][0]['id']
video_title = response['items'][0]['snippet']['title']
video_description = response['items'][0]['snippet']['description']
upload_date = response['items'][0]['snippet']['publishedAt'].replace('T', ' ').replace('Z', '')
duration = response['items'][0]['contentDetails']['duration'][2:].replace('M', ':').replace('S', '')
# query the response to assign data to corresponding variable

In [6]:
srt = YouTubeTranscriptApi.get_transcript(video_id, languages = ['fr'])
# get the captions data

In [7]:
transcript = ''
for i in srt:
    transcript += i['text']
# get just the text from the captions data and concatenate

In [2]:
# transcript = transcript.replace('\xa0\n', ' ')
# transcript = transcript.replace('\xa0\xa0', ' ')
# transcript = transcript.replace('\xa0', ' ')
# transcript = transcript.replace('\n', ' ')
# # clean the text if necessary, especially if it's intervening the charachters from which we will be splitting the text

In [20]:
all_interview = transcript.split('- ')
# gets a dialog list (assuming new speaker's caption starting with a dash)
# this substring can change , the text should be inspected

In [21]:
just_interviewer = []
just_candidate = []

for i in range(1, len(all_interview),2):
    just_interviewer.append(all_interview[i -1])
    just_candidate.append(all_interview[i])
# split the list into interviewer and candidate speech
# this assumes that the interviewer speaks first

In [22]:
df = pd.DataFrame({'candidate' : candidate,
                 'content_type': content_type,
                 'channel_id': channel_id,
                 'channel_title': channel_title,
                 'video_id' : video_id,
                 'video_title': video_title,
                 'video_description': video_description,
                 'upload_date' : upload_date,
                 'duration': duration, 
                 'interviewer_questions' : just_interviewer, 
                 'candidate_answers' : just_candidate })
#create the dataframe